In [1]:
# Last Revised - 2024.04.19
# Determine all the cells that are wet within a 2D HEC-RAS model

import h5py
from shapely.geometry import Point, LineString, Polygon, MultiPolygon
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.ops import unary_union

In [2]:
# Incoming from previous steps

# ----------------------
# Path to the HDF file
hdf_file_path = r'E:\HECRAS_2D_12070205\base_model_20240414_copy\BLE_LBSG_501.p04.hdf'

# ************
flt_firehose_flow = 14100
str_start_node = "wb-2410249"
flt_mainstem = 1884413.0
str_run_name = '1884413_wb-2410249_29-hr_14100-cfs'

dict_hec_info = {'HEC-RAS_Version': 'HEC-RAS 6.5 February 2024',
 'Project_Path': 'E:\\HECRAS_2D_12070205\\base_model_20240414_copy\\BLE_LBSG_501.prj',
 'Project_Title': 'LBSG_501',
 'Plan_Path': 'E:\\HECRAS_2D_12070205\\base_model_20240414_copy\\BLE_LBSG_501.p04',
 'Geometry_Path': 'E:\\HECRAS_2D_12070205\\base_model_20240414_copy\\BLE_LBSG_501.g04',
 'Geometry_HDF_Path': 'E:\\HECRAS_2D_12070205\\base_model_20240414_copy\\BLE_LBSG_501.g04.hdf',
 'Unsteady_File_Path': 'E:\\HECRAS_2D_12070205\\base_model_20240414_copy\\BLE_LBSG_501.u04',
 '2D_Flow_Area_Names': ['1207020501']}
# ************

# Output
# Specify the path where you want to save the GeoPackage file
output_path = r'E:\sample_2d_output\hydraulic_results_1884413_wb-2410249_29-hr_14100-cfs.gpkg'

# ++++++++++++++++
# run parameters
#
# input - nextgen hydropackage geopackage
gpkg_path = r'E:\ras2fim-2d\nextgen-test\nextgen_12.gpkg'

# number of iterations to buffer the nearest cells. 0 is just the nearest cells.
# 1 = the nearest cells plus the first cells touching those nearert... growing from there.
int_buffer_cells = 5

# Stable is a rolling average gradient of WSEL that is less than flt_max_allowed_gradient
flt_max_allowed_gradient = 0.009

int_len_gradient = 4 #number of time steps to get average gradient
# ++++++++++++++++

In [3]:
# ---------------------------------
def fn_compute_average_gradients(values, int_len_gradient):
    num_values = len(values)
    list_avg_gradient = []

    for i in range(num_values):
        if i < int_len_gradient:
            avg_gradient = np.nan
        else:
            gradient = (values[i] - values[i - int_len_gradient]) / int_len_gradient
            avg_gradient = gradient / int_len_gradient
            
        list_avg_gradient.append(avg_gradient)

    return list_avg_gradient
# ---------------------------------

In [4]:
# ------------------------
def fn_get_group_names(hdf5_file_path, group_path):
    """
    Retrieve the names of groups within a specified HDF5 file under a given group path.

    Parameters:
    hdf5_file_path (str): The file path to the HDF5 file.
    group_path (str): The path to the group whose subgroups' names are to be retrieved.

    Returns:
    list or None: A list containing the names of groups found under the specified group path. 
                  Returns None if the group path does not exist in the HDF5 file.
    """
    try:
        with h5py.File(hdf5_file_path, 'r') as hdf_file:
            # Check if the specified group path exists
            if group_path in hdf_file:
                group = hdf_file[group_path]

                # Extract names of HDF5 Group objects
                group_names = [name for name in group if isinstance(group[name], h5py.Group)]

                return group_names
            else:
                print(f"Group '{group_path}' not found in the HDF5 file.")
                return None
    except Exception as e:
        print(f"An error occurred: {e}")
# ------------------------

In [5]:
# Specify the HDF5 file path and group path
group_path = '/Geometry/2D Flow Areas/'

# Get names of HDF5 Group objects in the specified group
list_group_names = fn_get_group_names(hdf_file_path, group_path)
print(list_group_names)

# determine all the cells that are 'wet'

str_hdf_folder_results = '/Results/Unsteady/Output/Output Blocks/Base Output/Unsteady Time Series/2D Flow Areas/'
str_hdf_folder_results = str_hdf_folder_results + list_group_names[0] + '/'

str_cell_min_elev_path = group_path + list_group_names[0] + '/' + 'Cells Minimum Elevation'

# Open the HDF file
with h5py.File(hdf_file_path, 'r') as hdf_file:
    wsel_dataset_path = str_hdf_folder_results + 'Water Surface'
    wsel_data_per_cell = hdf_file[wsel_dataset_path][:]
    
    arr_min_elev_per_cell = hdf_file[str_cell_min_elev_path][:]
    
# Transpose the array
arr_wsel_data_per_cell_t = wsel_data_per_cell.T

# Identify nan values and replace them with zeros
#arr_min_elev_per_cell[np.isnan(arr_min_elev_per_cell)] = 0

# Subtract arr_min_elev_per_cell from each row of arr_wsel_data_per_cell_t
arr_depth_per_cell = arr_wsel_data_per_cell_t - arr_min_elev_per_cell[:, np.newaxis]

# Identify nan values and replace them with zeros
arr_depth_per_cell[np.isnan(arr_depth_per_cell)] = 0

list_indices_per_column = []

# Iterate over each column index
for column_index in range(arr_depth_per_cell.shape[1]):
    # Get values in the current column
    column_values = arr_depth_per_cell[:, column_index]
    
    # Find indices where values are greater than 0
    indices = np.where(column_values > 0)[0]
    
    list_indices_per_column.append(indices)

# Flatten the list of lists
flattened_indices = [index for sublist in list_indices_per_column for index in sublist]

# Get unique indices
unique_indices = np.unique(flattened_indices)
list_unique_indices_sorted = np.sort(unique_indices)

['1207020501']


In [6]:
# Filter the array to those cells that are wet
arr_depth_wet_cells = arr_depth_per_cell[list_unique_indices_sorted]

# Round all values to two decimal points
arr_depth_wet_cells = np.round(arr_depth_wet_cells, 2)

In [7]:
# Initialize an empty list to store the results
list_results = []

for row in arr_depth_wet_cells:
    list_result = fn_compute_average_gradients(row, int_len_gradient)
    
    # Append the result to the list of results
    list_results.append(list_result)
    
# Convert the list of lists into a numpy array
arr_list_wsel_gradient = np.array(list_results)

In [8]:
# get the shapefile of all the cells in unique_indices

str_hdf_folder_2darea = group_path + list_group_names[0] + '/'
hdf5_file_path = hdf_file_path

# Location of Face Point Coordinates in HDF5
str_facepoint_coords = str_hdf_folder_2darea + 'FacePoints Coordinate'

# Open the HDF5 file
with h5py.File(hdf5_file_path, 'r') as hdf_file:
    # Extract X and Y coordinates
    x_coordinates = hdf_file[str_facepoint_coords][:, 0]
    y_coordinates = hdf_file[str_facepoint_coords][:, 1]

# Create a pandas DataFrame
df_facepoints = pd.DataFrame({'X': x_coordinates, 'Y': y_coordinates})

# Location of Indices of face points making up the cells
str_cells_facepoint_indexes = str_hdf_folder_2darea + 'Cells FacePoint Indexes'

# Open the HDF5 file
with h5py.File(hdf5_file_path, 'r') as hdf_file:
    # Extract face points coordinate data
    facepoints_data = hdf_file[str_cells_facepoint_indexes][:]

    # Extract the projection
    projection_wkt = hdf_file.attrs['Projection'].decode('utf-8')

# Create a GeoDataFrame to store the polygons
geometry = []
indices = []

for row_idx, row in enumerate(facepoints_data):
    polygon_coords = []
    
    if row_idx in list_unique_indices_sorted:
        for idx in row:
            if idx != -1:
                x = df_facepoints.loc[idx, 'X']
                y = df_facepoints.loc[idx, 'Y']
                polygon_coords.append((x, y))
        # Check if the polygon has at least 3 points (needed to create a polygon)
        if len(polygon_coords) >= 3:
            # Connect to the first point to close the polygon
            polygon_coords.append(polygon_coords[0])
            geometry.append(Polygon(polygon_coords))
            indices.append(row_idx)  # Append the row index as the cell index

# Create a GeoDataFrame
gdf_cells = gpd.GeoDataFrame(geometry=geometry, index=indices, columns=['geometry'], crs=projection_wkt)

# create a new coloumn that contains the cell index
gdf_cells['cell_idx'] = gdf_cells.index

In [9]:
arr_depth_wet_cells_nan = arr_depth_wet_cells

for row_idx in range(arr_depth_wet_cells_nan.shape[0]):
    row = arr_depth_wet_cells_nan[row_idx]
    
    # Find the index of the first non-zero element in the row
    first_non_zero_index = np.argmax(row != 0)
    
    # Set all elements before the first non-zero element to NaN
    row[:first_non_zero_index] = np.nan

# Finding locations where values are nan in arr_depth_wet_cells_nan
arr_nan_indices = np.isnan(arr_depth_wet_cells_nan)

# Setting values to nan in arr_list_wsel_gradient at corresponding locations
arr_list_wsel_gradient[arr_nan_indices] = np.nan

In [10]:
arr_wsel_gradient_stability = arr_list_wsel_gradient.copy()

# Using list comprehension to get absolute values of floats while ignoring nan
arr_wsel_gradient_stability = np.abs(arr_wsel_gradient_stability)

In [11]:
column_names = [f'wsel_grad_t{i+1:03d}' for i in range(len(arr_wsel_gradient_stability[0]))]

# Convert arr_list_wsel_gradient to a DataFrame with the specified column names
df_wsel_grad = pd.DataFrame(arr_wsel_gradient_stability, columns=column_names)

# Add list_unique_indices_sorted as a new column named 'unique_indices_sorted'
df_wsel_grad['cell_idx'] = list_unique_indices_sorted

In [12]:
gdf_wsel_gradient = pd.merge(gdf_cells, df_wsel_grad, on='cell_idx', how='left')

In [13]:
# Assuming gdf_wsel_gradient is your GeoDataFrame
#output_path = r'E:\working\export_wsel_gradient_values_ar.gpkg'

# Export GeoDataFrame to a GeoPackage
#gdf_wsel_gradient.to_file(output_path, driver='GPKG')

In [14]:
# --------------
# Added 2024.04.14
# Determine the index of the first stable cell calcaulation

# Initialize an empty list to store the positions
arr_positions = []

# Iterate over each row
for row in arr_wsel_gradient_stability:
    # Find the first non-NaN value that is less than 0.009
    
    index = np.where(~np.isnan(row) & (row < flt_max_allowed_gradient))[0]
    if index.size > 0:
        arr_positions.append(index[0])
    else:
        arr_positions.append(None)

# Convert the list to a numpy array
arr_positions = np.array(arr_positions)

# Replace 'None' with -1
arr_positions = np.where(arr_positions == None, -1, arr_positions)

# Convert positions to integers
arr_positions_int = arr_positions.astype(int)

print("Positions of the first value where not NaN and less than 0.009 for each row:")
print(arr_positions_int)

# Creating DataFrame
df_hours_to_stability = pd.DataFrame({'cell_idx': list_unique_indices_sorted, 'hours_to_stable': arr_positions_int})

# Create a gdf of cells with "hours_to_stability"
# Note that an unstable cells is -1
gdf_hours_to_stable = pd.merge(gdf_cells, df_hours_to_stability, on='cell_idx', how='left')

Positions of the first value where not NaN and less than 0.009 for each row:
[12  8  5 ... 18 18 18]


In [15]:
# from NextGen hydrofabric - get all the streamms where the 'mainstem == flt_mainstem' Example: 1884894.0

# Load the GeoPackage layers
gdf_flowpaths = gpd.read_file(gpkg_path, layer='flowpaths')
gdf_nexus = gpd.read_file(gpkg_path, layer='nexus')

# find all the reaches that have a 'mainstem' == flt_mainstem
gdf_selected_mainstem_streams = gdf_flowpaths[gdf_flowpaths['mainstem'] == flt_mainstem]

# Set the crs of the mainstem to crs of cells
gdf_selected_mainstem_streams = gdf_selected_mainstem_streams.to_crs(gdf_cells.crs)

# Compute the centroid for every cell's polygon
gdf_cells['geom_centroid'] = gdf_cells.geometry.centroid

In [16]:
# ----------
def fn_find_nearest_flowpath_and_distance(centroid, gdf_mainstems):
    # Calculate the distances from the centroid to all mainstem lines
    distances = gdf_mainstems.distance(centroid)
    # Find the index of the minimum distance
    int_nearest_index = distances.idxmin()
    flt_nearest_distance = distances.min()
    
    str_nearest_id = gdf_mainstems.loc[int_nearest_index]['id']
    
    return str_nearest_id, flt_nearest_distance
# ----------

In [17]:
# find the most downstream mainstem
gdf_largest_drainage_area = gdf_selected_mainstem_streams.loc[gdf_selected_mainstem_streams['tot_drainage_areasqkm'].idxmax()]
str_toid_with_largest_drainage_area = gdf_largest_drainage_area['toid']
str_toid_with_largest_drainage_area

gdf_selected_rows = gdf_nexus[gdf_nexus['id'] == str_toid_with_largest_drainage_area]

In [18]:
if len(gdf_selected_rows) == 1:
    # only one downstream node found
    str_dwn_stream_id = gdf_selected_rows.iloc[0]['toid']
    
    # Find the stream in gdf_flowpaths where 'id' == str_dwn_stream_id
    
    # Finding the stream
    gdf_downstream_stream = gdf_flowpaths[gdf_flowpaths['id'] == str_dwn_stream_id]
    
    # Set the crs of the mainstem to crs of cells
    gdf_downstream_stream = gdf_downstream_stream.to_crs(gdf_cells.crs)
    
    # Concatenating DataFrames
    gdf_all_streams = pd.concat([gdf_downstream_stream, gdf_selected_mainstem_streams])
else:
    gdf_all_streams = gdf_selected_mainstem_streams
    
    
# Apply the function to each centroid in gdf_cells
nearest_info = gdf_cells['geom_centroid'].apply(lambda x: fn_find_nearest_flowpath_and_distance(x, gdf_all_streams))

gdf_cells['nearest_flowpath'] = nearest_info.apply(lambda x: x[0])
gdf_cells['distance_to_nearest_flowpath'] = nearest_info.apply(lambda x: x[1])

In [19]:
# List of unique streams in gdf_all_streams
list_unique_stream_ids = gdf_all_streams['id'].unique().tolist()

# remove the downstream stream
# we don't want this polygons as it is on another river
list_unique_stream_ids.remove(str_dwn_stream_id)

# For each item in list_unique_stream_ids, create polygons of the disolved cells in gdf_cells.  Disolve on 'nearest_flowpath'

list_dissolved_polygons_gdf = []

for stream_id in list_unique_stream_ids:
    # Filter cells associated with the current stream ID
    cells_for_stream = gdf_cells[gdf_cells['nearest_flowpath'] == stream_id]
    
    # Dissolve cells based on the 'nearest_flowpath' column
    dissolved = cells_for_stream.dissolve(by='nearest_flowpath')
    
    # Drop specified columns
    dissolved.drop(columns=['cell_idx', 'distance_to_nearest_flowpath'], inplace=True)

    # Append the dissolved polygon to the list
    list_dissolved_polygons_gdf.append(dissolved)


# There can be streams along the mainsteam that have no geometry.  This is because the 
# 'firehose' is downstream of these reach or the water never got to that reach.
# Drop those reaches without geometry from list_dissolved_polygons_gdf

# Remove reaches without geometry from list_dissolved_polygons_gdf
list_dissolved_polygons_gdf_filtered = []

for gdf_merged_poly in list_dissolved_polygons_gdf:
    if not gdf_merged_poly.empty:
        list_dissolved_polygons_gdf_filtered.append(gdf_merged_poly)
        
list_dissolved_polygons_gdf = list_dissolved_polygons_gdf_filtered

list_buffered_shp_polys = []
list_mainstem = []

for gdf_merged_poly in list_dissolved_polygons_gdf:
    
    str_mainstem = gdf_merged_poly.index[0]
    list_mainstem.append(str_mainstem)
    
    shp_polygon_to_check = gdf_merged_poly['geometry'].iloc[0]

    for i in range(int_buffer_cells):
        gdf_intersecting_cells = gdf_cells[gdf_cells.geometry.touches(shp_polygon_to_check)]
        merged_geometry = unary_union(gdf_intersecting_cells.geometry)
        merged_polygon = shp_polygon_to_check.union(merged_geometry)
        shp_polygon_to_check = merged_polygon

    list_buffered_shp_polys.append(shp_polygon_to_check)

In [20]:
# Create a GeoDataFrame from the list of shapely objects and list of strings
gdf_buffered_limits = gpd.GeoDataFrame({'flowpath': list_mainstem, 'geometry': list_buffered_shp_polys})

# Set the crs of the newly created geodataframe
gdf_buffered_limits.crs =  gdf_cells.crs

In [21]:
# merge gdf_buffered_limits into a single polygon

# Assuming your GeoDataFrame is named gdf and contains a column 'geometry' with multiple polygons

# First, ensure that the 'geometry' column contains Polygon or MultiPolygon objects
# This is usually done when reading the GeoDataFrame, but just in case
gdf_buffered_limits['geometry'] = gdf_buffered_limits['geometry'].apply(lambda geom: geom if isinstance(geom, MultiPolygon) else MultiPolygon([geom]))

# Then, merge all the polygons into a single MultiPolygon
merged_geometry = gdf_buffered_limits['geometry'].unary_union

# Create a new GeoDataFrame with the merged geometry
gdf_merged_flooeded_cells = gpd.GeoDataFrame(geometry=[merged_geometry])

# Set the crs of the newly created geodataframe
gdf_merged_flooeded_cells.crs =  gdf_buffered_limits.crs

In [22]:
# Clip gdf_cells using gdf_merged_flooded_cells geometry
gdf_cells_clipped = gpd.clip(gdf_cells, gdf_merged_flooeded_cells.geometry)

C:\Users\civil\anaconda3\envs\tx-bridge\lib\site-packages\geopandas\tools\clip.py:67: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  clipped.loc[


In [23]:
# Select columns from gdf_hours_to_stable excluding 'geometry'
gdf_hours_to_stable_subset = gdf_hours_to_stable.drop(columns=['geometry'])

# Drop coloumns from gdf_cells_clipped
gdf_cells_clipped_subset = gdf_cells_clipped.drop(columns=['geom_centroid', 'distance_to_nearest_flowpath'])

# Perform left join
gdf_cells_clipped_w_hour = pd.merge(gdf_cells_clipped_subset, gdf_hours_to_stable_subset, on='cell_idx', how='left')

In [24]:
# from the array of WSEL, get the stable timestep's water surface elevation for each cell
list_wsel_per_cell = []

for index,row in gdf_cells_clipped_w_hour.iterrows():
    x = row['cell_idx']
    y = row['hours_to_stable']
    # Note - if not stable, value is set as last time step... y=-1
    
    list_wsel_per_cell.append(arr_wsel_data_per_cell_t[x][y])
    
gdf_cells_wsel = gdf_cells_clipped_w_hour.copy()

gdf_cells_wsel['wsel'] = list_wsel_per_cell

In [25]:
# Save a geodataframe of the gdf_cells_wsel and gdf_buffered_limits

In [26]:
# Document the constant flow used in this simulation
gdf_cells_wsel['flow_cfs'] = flt_firehose_flow

gdf_cells_wsel['run_name'] = str_run_name

# Correct the hours to stable... this is a moving average, so subtract this moving average to get to
# correct travel time for the given flow.

# Subtract int_len_gradient from 'hours_to_stable' column in every row
gdf_cells_wsel['hours_to_stable'] = gdf_cells_wsel['hours_to_stable'] - int_len_gradient

# Replace values in 'hours_to_stable' column <= 0 with -1
# to preserve the 'unstable' value
gdf_cells_wsel.loc[gdf_cells_wsel['hours_to_stable'] <= 0, 'hours_to_stable'] = -1

# if gdf_cells_wsel contains any geomety that is not Polygon, drop those rows
gdf_cells_wsel = gdf_cells_wsel[gdf_cells_wsel.geometry.geom_type == 'Polygon']

In [27]:
# How many cells was this boundary buffered out
gdf_buffered_limits['buffer_cell_count'] = int_buffer_cells

# Document the constant flow used in this simulation
gdf_buffered_limits['flow_cfs'] = flt_firehose_flow

gdf_buffered_limits['run_name'] = str_run_name

In [28]:
gdf_buffered_limits

,flowpath,geometry,buffer_cell_count,flow_cfs,run_name
0,wb-2410251,"MULTIPOLYGON (((2992649.306 10278157.295, 2992...",5,14100,1884413_wb-2410249_29-hr_14100-cfs
1,wb-2410255,"MULTIPOLYGON (((3027376.471 10269306.529, 3027...",5,14100,1884413_wb-2410249_29-hr_14100-cfs
2,wb-2410258,"MULTIPOLYGON (((3052642.639 10239632.368, 3052...",5,14100,1884413_wb-2410249_29-hr_14100-cfs
3,wb-2410259,"MULTIPOLYGON (((3055911.199 10237312.998, 3055...",5,14100,1884413_wb-2410249_29-hr_14100-cfs
4,wb-2410260,"MULTIPOLYGON (((3065036.132 10228181.509, 3064...",5,14100,1884413_wb-2410249_29-hr_14100-cfs
5,wb-2410261,"MULTIPOLYGON (((3079876.910 10224539.263, 3079...",5,14100,1884413_wb-2410249_29-hr_14100-cfs
6,wb-2410249,"MULTIPOLYGON (((2954641.059 10296841.940, 2954...",5,14100,1884413_wb-2410249_29-hr_14100-cfs
7,wb-2410250,"MULTIPOLYGON (((2978767.625 10288850.487, 2978...",5,14100,1884413_wb-2410249_29-hr_14100-cfs
8,wb-2410252,"MULTIPOLYGON (((3001776.471 10275506.529, 3001...",5,14100,1884413_wb-2410249_29-hr_14100-cfs
9,wb-2410253,"MULTIPOLYGON (((3007176.471 10273906.529, 3006...",5,14100,1884413_wb-2410249_29-hr_14100-cfs


In [29]:
# convert the gdf_buffered_limits of multipolygons to polygons, if necessary.  New row for each polygon

# Create an empty list to store the new rows
new_rows = []

# Iterate over each row in the GeoDataFrame
for index, row in gdf_buffered_limits.iterrows():
    # Check if the geometry is a MultiPolygon
    if row['geometry'].geom_type == 'MultiPolygon':
        # If it is, iterate over each polygon in the MultiPolygon
        for polygon in row['geometry']:
            # Create a new row with the same attributes but with a single Polygon geometry
            new_row = row.drop('geometry').copy()
            new_row['geometry'] = polygon
            new_rows.append(new_row)
    else:
        # If it's already a Polygon, just append the original row
        new_rows.append(row)

# Create a new GeoDataFrame from the list of new rows
gdf_buffered_polygon = gpd.GeoDataFrame(new_rows, crs=gdf_buffered_limits.crs)

C:\Users\civil\AppData\Local\Temp\ipykernel_9644\2819663173.py:11: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for polygon in row['geometry']:
C:\Users\civil\AppData\Local\Temp\ipykernel_9644\2819663173.py:11: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for polygon in row['geometry']:
C:\Users\civil\AppData\Local\Temp\ipykernel_9644\2819663173.py:11: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for polygon in row['geometry']:
C:\Users\civil\AppData\Local\Temp\ipykernel_9644\2819663173.py:11: ShapelyDeprecationWarning: Iteration ov

In [30]:
gdf_buffered_polygon

,flowpath,buffer_cell_count,flow_cfs,run_name,geometry
0,wb-2410251,5,14100,1884413_wb-2410249_29-hr_14100-cfs,"POLYGON ((2992649.306 10278157.295, 2992576.47..."
0,wb-2410251,5,14100,1884413_wb-2410249_29-hr_14100-cfs,"POLYGON ((2989882.070 10278412.128, 2989880.57..."
1,wb-2410255,5,14100,1884413_wb-2410249_29-hr_14100-cfs,"POLYGON ((3027376.471 10269306.529, 3027176.47..."
2,wb-2410258,5,14100,1884413_wb-2410249_29-hr_14100-cfs,"POLYGON ((3052642.639 10239632.368, 3052470.49..."
3,wb-2410259,5,14100,1884413_wb-2410249_29-hr_14100-cfs,"POLYGON ((3055911.199 10237312.998, 3055790.65..."
4,wb-2410260,5,14100,1884413_wb-2410249_29-hr_14100-cfs,"POLYGON ((3065036.132 10228181.509, 3064839.22..."
5,wb-2410261,5,14100,1884413_wb-2410249_29-hr_14100-cfs,"POLYGON ((3079876.910 10224539.263, 3079734.02..."
6,wb-2410249,5,14100,1884413_wb-2410249_29-hr_14100-cfs,"POLYGON ((2954641.059 10296841.940, 2954644.95..."
7,wb-2410250,5,14100,1884413_wb-2410249_29-hr_14100-cfs,"POLYGON ((2978767.625 10288850.487, 2978608.88..."
8,wb-2410252,5,14100,1884413_wb-2410249_29-hr_14100-cfs,"POLYGON ((3001776.471 10275506.529, 3001725.46..."


In [31]:
# get the 'time to stable' for each buffered flowpath, the count of cells in the area
# and the count of unstable cells in the area

# Create empty lists to store results
highest_hours = []
lowest_hours = []
cell_count = []
negative_one_count = []

# Iterate over each row in gdf_buffered_polygon
for index, row in gdf_buffered_polygon.iterrows():
    # Intersect the polygon geometry with gdf_cells_wsel
    intersected_cells = gdf_cells_wsel[gdf_cells_wsel.intersects(row['geometry'])]
    int_cell_count = len(intersected_cells)
    
    # If there are intersected cells
    if not intersected_cells.empty:
        # Calculate highest and lowest hours_to_stable
        highest_hour = intersected_cells['hours_to_stable'].max()
        lowest_hour = intersected_cells['hours_to_stable'].min()
        # Count the number of cells where the value is -1
        negative_one_count_value = (intersected_cells['hours_to_stable'] == -1).sum()
    else:
        # If no intersection, set to None
        highest_hour = None
        lowest_hour = None
        negative_one_count_value = None
        
    
    # Append results to lists
    highest_hours.append(highest_hour)
    lowest_hours.append(lowest_hour)
    cell_count.append(int_cell_count)
    negative_one_count.append(negative_one_count_value)

# Add the highest and lowest hours, cell count, and negative one count to the original GeoDataFrame
gdf_buffered_polygon['highest_hours_to_stable'] = highest_hours
gdf_buffered_polygon['lowest_hours_to_stable'] = lowest_hours
gdf_buffered_polygon['cell_count'] = cell_count
gdf_buffered_polygon['unstable_cell_count'] = negative_one_count

In [32]:
# --- Create layer of streams computed, including percentage of stream line that is 'wet' and 'stable' ---
# Create a geodataframe of the streams that are in list_unique_stream_ids

# Select rows where 'id' column is in list_unique_stream_ids
gdf_streams_in_simulation = gdf_all_streams[gdf_all_streams['id'].isin(list_unique_stream_ids)]

# cells with only the 'hours_to_stable' attribute
gdf_cells_wsel_light = gdf_cells_wsel[['geometry', 'hours_to_stable']].copy()

gdf_streams_in_simulation_light = gdf_streams_in_simulation[['geometry','id', 'mainstem',
                                                             'order','tot_drainage_areasqkm']].copy()

# Perform geometric overlay to compute intersection
gdf_cell_intersection = gpd.overlay(gdf_streams_in_simulation_light, gdf_cells_wsel_light, how='intersection')

# Calculate length of each line
gdf_cell_intersection['segment_length'] = gdf_cell_intersection.geometry.length

# Calculte length of the entire stream reach
gdf_streams_in_simulation_light['length'] = gdf_streams_in_simulation_light.geometry.length

# create pandas series that is sum of segment_length by unique 'id'
ps_sum_segment_length = gdf_cell_intersection.groupby('id')['segment_length'].sum()

# create pandas series that is sum of segment_length by unique 'id' where the stream is 'stable'
filtered_df = gdf_cell_intersection[gdf_cell_intersection['hours_to_stable'] > 0]
ps_sum_segment_length_stable = filtered_df.groupby('id')['segment_length'].sum()

# Compute the percent of the stream that is 'wet' and the percentrage that the stream is 'stable'

# Merge the GeoDataFrame with the pandas Series based on the 'id' column
merged_df = gdf_streams_in_simulation_light.merge(ps_sum_segment_length_stable.rename('wet_length'), left_on='id', right_index=True)

# Merge the GeoDataFrame with the pandas Series based on the 'id' column
gdf_streams_w_stats = merged_df.merge(ps_sum_segment_length_stable.rename('stable_length'), left_on='id', right_index=True)

# Compute percentage of stable length
gdf_streams_w_stats['perct_wet'] = round((gdf_streams_w_stats['wet_length'] / gdf_streams_w_stats['length']) * 100,1)

# Compute percentage of stable length
gdf_streams_w_stats['perct_stable'] = round((gdf_streams_w_stats['stable_length'] / gdf_streams_w_stats['length']) * 100,1)

# Add run name to streams 
list_unique_run = gdf_cells_wsel['run_name'].unique().tolist()
gdf_streams_w_stats['run_name'] = list_unique_run[0] # Assumes there is only one value

In [33]:
# Create a dataframe from the HEC-RAS run info
df_hec_info = pd.DataFrame(dict_hec_info) 

# Create a GeoDataFrame with a geometry column set to None
# TODO - 2024.04.16 = for now this is a table at 0,0 as point... revise to SQLite table (no spatial data).
gdf_hec_info = gpd.GeoDataFrame(df_hec_info, geometry=[Point(0, 0)] * len(df_hec_info))

# Write GeoDataFrames to GeoPackage as separate layers
gdf_streams_w_stats.to_file(output_path, layer='03_streams_ln', driver="GPKG")
gdf_cells_wsel.to_file(output_path, layer='02_cells_wsel_ar', driver="GPKG")
gdf_buffered_polygon.to_file(output_path, layer='01_flowpath_flooded_cells_ar', driver="GPKG")

gdf_hec_info.to_file(output_path, layer='00_hec_info', driver="GPKG")